__качаем объявления о продаже недвижимости с avito.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sevastopol/kvartiry/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sevastopol/zemelnye_uchastki/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sevastopol/doma_dachi_kottedzhi/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sudak/kvartiry/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sudak/zemelnye_uchastki/prodam',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sudak/doma_dachi_kottedzhi/prodam',
    },
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://www.avito.ru/bahchisaray/kvartiry/prodam',
#     }, 
    {
            'place': 'bahchisaray',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/bahchisaray/zemelnye_uchastki/prodam',
    },{
            'place': 'bahchisaray',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam',
    },

]

---

In [4]:
# from selenium import webdriver
# from lib.avito import AvitoPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = AvitoPageParser().transform(browser)
# # browser.close()

In [5]:
# browser.close()

In [6]:
# data

---

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.avito import AvitoParser

In [8]:
dest_folder = 'data/2024/raw'

In [9]:
%%time 


options = webdriver.FirefoxOptions()
#options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    sleep(3)
    
    data = AvitoParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place

    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/avito/avito_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )

browser.close()

[INFO    ] 2024-07-08 13:13:03 | open virtual browser


'https://www.avito.ru/sevastopol/kvartiry/prodam'

[INFO    ] 2024-07-08 13:13:16 | page 1: 53 items
[INFO    ] 2024-07-08 13:13:27 | page 2: 50 items
[INFO    ] 2024-07-08 13:13:40 | page 3: 50 items
[INFO    ] 2024-07-08 13:13:51 | page 4: 50 items
[INFO    ] 2024-07-08 13:14:00 | page 5: 50 items
[INFO    ] 2024-07-08 13:14:10 | page 6: 50 items
[INFO    ] 2024-07-08 13:14:19 | page 7: 50 items
[INFO    ] 2024-07-08 13:14:29 | page 8: 50 items
[INFO    ] 2024-07-08 13:14:39 | page 9: 50 items
[INFO    ] 2024-07-08 13:14:50 | page 10: 50 items
[INFO    ] 2024-07-08 13:15:00 | page 11: 50 items
[INFO    ] 2024-07-08 13:15:09 | page 12: 50 items
[INFO    ] 2024-07-08 13:15:19 | page 13: 50 items
[INFO    ] 2024-07-08 13:15:30 | page 14: 50 items
[INFO    ] 2024-07-08 13:15:40 | page 15: 50 items
[INFO    ] 2024-07-08 13:15:49 | page 16: 50 items
[INFO    ] 2024-07-08 13:16:00 | page 17: 50 items
[INFO    ] 2024-07-08 13:16:11 | page 18: 50 items
[INFO    ] 2024-07-08 13:16:21 | page 19: 50 items
[INFO    ] 2024-07-08 13:16:31 | page 20

4414

'data/2024/raw/flat/avito/avito_2024-07-08_13-29_raw_sevastopol.pkl'

...

'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam'

[INFO    ] 2024-07-08 13:48:43 | page 1: 50 items
[INFO    ] 2024-07-08 13:48:52 | page 2: 50 items
[INFO    ] 2024-07-08 13:49:02 | page 3: 50 items
[INFO    ] 2024-07-08 13:49:11 | page 4: 50 items
[INFO    ] 2024-07-08 13:49:21 | page 5: 50 items
[INFO    ] 2024-07-08 13:49:26 | page 6: 24 items


274

'data/2024/raw/house/avito/avito_2024-07-08_13-49_raw_bahchisaray.pkl'

CPU times: user 4min 44s, sys: 11.3 s, total: 4min 55s
Wall time: 36min 25s


In [10]:
# browser.close()

In [11]:
!date

Пн 08 июл 2024 13:49:27 MSK
